- Initialization:
    - Importing libraries and setting up constants for simulation.
- RCS Dynamics:
    - Calculating torques generated by thrusters.
- Grid Fin Dynamics:
    - Calculating aerodynamic forces (lift and drag).
- Transition Logic:
    - Determining when to switch control modes between RCS and grid fins.
- Simulation Loop:
    - Time-stepped integration of dynamics, updating altitude, velocity, and control states.
- Visualization:
    - Plotting altitude, torque, and angular velocity against time for analysis.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg

# --- Assumptions and Constants ---
G = 9.81  # Gravitational acceleration (m/s^2)
THRUST_POWER = 10  # Thrust power of RCS thrusters (N)
FIN_AREA = 0.1  # Grid fin area (m^2)
AIR_DENSITY_SEA_LEVEL = 1.225  # Air density at sea level (kg/m^3)
ALTITUDE_THRESHOLD_RCS = 30000  # Altitude for RCS dominance (m)
ALTITUDE_THRESHOLD_FINS = 20000  # Altitude for Grid Fins dominance (m)
DT = 0.1  # Time step for simulation (s)
TARGET_ALTITUDE = 0  # Target landing altitude (m)

# --- Initial Conditions ---
time_steps = 2000  # Number of simulation steps
altitude = 35000  # Starting altitude (m)
velocity = 300  # Vertical descent velocity (m/s)
angular_velocity = np.zeros(3, dtype=float)  # Initial angular velocity (rad/s)
fuel_used = 0  # Initialize fuel usage
perturbation_strength = 0.1  # Base intensity of random perturbations
torque = np.zeros(3, dtype=float)  # Initialize torque as float array

# --- Assumption ---
# 1. Constant descent velocity is assumed for simplicity.
# 2. Perturbations represent crosswinds or system noise with random and periodic dynamics.
# 3. RCS and Grid Fins are modeled separately, with Hybrid combining their effects during transition.
